# 自転車犯罪マップ

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/crime map.png" width=600>

このラボの目的:

- オープンデータソースからデータを取得し、Python ノートブックにインポート
- 複数の列から単一の住所フィールドを作成
- 緯度と経度の座標を割り当てて各行をジオコーディング
- 美しいチャートや地図を作成

## ライブラリをインポートする

このラボで使うライブラリーを一気にインポートしよう。

In [57]:
## for spatial analysis
import geopandas as gpd

## for data analysis
import pandas as pd

## for pretty charts
import plotly.express as px

# for plotly themes
import plotly.io as pio

## for URL requests
import urllib.request
import requests

## for maps
import folium
from folium import plugins

# オープンデータとは？

どんなプロジェクトでも、どんなマップでもデータが必要です。でも「いい」データって意外となかったりするので、イメージしていたマップが作れないことがよくある。そこで、最近政府機関がオープンデータを提供する方向性があり、あらゆる行政がCSVやEXCELフォーマットでデータをダウンロードできるように提供している。

Let's find some open data!

Here is an example:

https://www.pref.chiba.lg.jp/shoufuku/opendata/techoutoukei.html

## 自転車盗難データをダウンロード

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/Chiba%20police.png" width=400>

まずは千葉県警察のサイトにアクセス。そこから次に手順でデータをダウンロード：

➡️ https://www.police.pref.chiba.jp/

➡️ 安全な暮らし

➡️ 地域の防犯

➡️ あなたの町の犯罪情勢

➡️ オープンデータ

➡️ 自転車盗（CSV）

ダウンロードしたファイルを `chibike.csv` と名付けて、このフォルダー（Week08) にセーブ。

In [78]:
# ダウンロードしたデータを読み込もう
df = pd.read_csv('chibike.csv')

In [73]:
# データの情報
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7346 entries, 0 to 7345
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             7346 non-null   object
 1   手口             7346 non-null   object
 2   管轄警察署（発生地）     7346 non-null   object
 3   管轄交番・駐在所（発生地）  7343 non-null   object
 4   市区町村コード（発生地）   7346 non-null   int64 
 5   都道府県（発生地）      7346 non-null   object
 6   市区町村（発生地）      7346 non-null   object
 7   町丁目（発生地）       7330 non-null   object
 8   発生年月日（始期）      7346 non-null   object
 9   発生時（始期）        7346 non-null   object
 10  発生場所           7346 non-null   object
 11  発生場所の詳細        7346 non-null   object
 12  被害者の年齢         7346 non-null   object
 13  被害者の職業         7346 non-null   object
 14  施錠関係           7346 non-null   object
dtypes: int64(1), object(14)
memory usage: 861.0+ KB


Wow! ７千件もある！ちょっと多いので場所で絞りましょう。<h1>「柏」</h1>だけのデータフレームを作ろう。

In [79]:
df

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,被害者の年齢,被害者の職業,施錠関係
0,窃盗,自転車盗,千葉中央,京成千葉中央駅前交番,121011,千葉県,千葉市中央区,新宿１丁目,2022-02-10,8,その他,駐車（輪）場,50歳代,その他,施錠せず
1,窃盗,自転車盗,千葉中央,京成千葉中央駅前交番,121011,千葉県,千葉市中央区,新宿１丁目,2022-03-22,20,その他の住宅（３階建て以下共同住宅等）,駐車（輪）場,20歳代,その他,施錠した
2,窃盗,自転車盗,千葉中央,京成千葉中央駅前交番,121011,千葉県,千葉市中央区,新宿１丁目,2022-11-06,18,４階建て以上共同住宅,駐車（輪）場,30歳代,その他,施錠した
3,窃盗,自転車盗,千葉中央,京成千葉中央駅前交番,121011,千葉県,千葉市中央区,新宿２丁目,2022-03-01,21,４階建て以上共同住宅,駐車（輪）場,40歳代,その他,施錠した
4,窃盗,自転車盗,千葉中央,京成千葉中央駅前交番,121011,千葉県,千葉市中央区,新宿２丁目,2022-04-07,9,その他,その他,40歳代,その他,施錠した
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7341,窃盗,自転車盗,四街道,旭交番,122289,千葉県,四街道市,みそら２丁目,2022-09-10,13,その他,駐車（輪）場,10歳代,中学生,施錠せず
7342,窃盗,自転車盗,四街道,旭交番,122289,千葉県,四街道市,みそら３丁目,2022-10-26,8,一戸建住宅,駐車（輪）場,10歳代,高校生,施錠せず
7343,窃盗,自転車盗,四街道,旭交番,122289,千葉県,四街道市,山梨,2022-02-15,21,一戸建住宅,その他,10歳代,中学生,施錠せず
7344,窃盗,自転車盗,四街道,旭交番,122289,千葉県,四街道市,吉岡,2022-10-30,14,道路上,その他,70歳以上,その他,施錠せず


In [80]:
kashiwa = df[df['管轄警察署（発生地）'] == '柏'].copy()

In [81]:
# データをチェック
kashiwa.sample(5)

,罪名,手口,管轄警察署（発生地）,管轄交番・駐在所（発生地）,市区町村コード（発生地）,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,発生年月日（始期）,発生時（始期）,発生場所,発生場所の詳細,被害者の年齢,被害者の職業,施錠関係
2993,窃盗,自転車盗,柏,旭町交番,122173,千葉県,柏市,旭町１丁目,2022-10-24,19,その他,その他,10歳代,中学生,施錠した
3243,窃盗,自転車盗,柏,高柳交番,122173,千葉県,柏市,高柳１丁目,2022-09-23,9,その他,駐車（輪）場,10歳代,高校生,施錠せず
3121,窃盗,自転車盗,柏,柏駅前交番,122173,千葉県,柏市,柏３丁目,2022-11-11,18,４階建て以上共同住宅,駐車（輪）場,20歳代,その他,施錠した
2869,窃盗,自転車盗,柏,南増尾交番,122173,千葉県,柏市,青葉台２丁目,2022-09-22,19,その他,駐車（輪）場,10歳代,中学生,施錠せず
2782,窃盗,自転車盗,柏,光ヶ丘交番,122173,千葉県,柏市,酒井根３丁目,2022-11-13,21,一戸建住宅,駐車（輪）場,10歳代,高校生,施錠せず


In [82]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 471 entries, 2776 to 3246
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             471 non-null    object
 1   手口             471 non-null    object
 2   管轄警察署（発生地）     471 non-null    object
 3   管轄交番・駐在所（発生地）  471 non-null    object
 4   市区町村コード（発生地）   471 non-null    int64 
 5   都道府県（発生地）      471 non-null    object
 6   市区町村（発生地）      471 non-null    object
 7   町丁目（発生地）       471 non-null    object
 8   発生年月日（始期）      471 non-null    object
 9   発生時（始期）        471 non-null    object
 10  発生場所           471 non-null    object
 11  発生場所の詳細        471 non-null    object
 12  被害者の年齢         471 non-null    object
 13  被害者の職業         471 non-null    object
 14  施錠関係           471 non-null    object
dtypes: int64(1), object(14)
memory usage: 58.9+ KB


### `value_counts`でチャート

このデータを見て、咄嗟に知りたいものってなんでしょう？

例えば、<h1>「1日の何時に自転車の盗難が一番発生するの？」</h1>の質問に答えるためにはどのようなデータ分析が必要でしょうか？

では、そのチャートを作ってみましょう。

データには **【発生時（始期）】** のカラムがあるので、各時間帯のカウントを`value_counts()`で調べる。最後に足される`reset_index()`で結果をデータフレームに変換します。

In [83]:
# create a new variable with hourly counts
time = kashiwa['発生時（始期）'].value_counts().reset_index()

time

,発生時（始期）,count
0,18,44
1,19,39
2,8,37
3,16,36
4,20,33
5,17,32
6,12,28
7,13,25
8,15,24
9,7,23


この結果は良いが、カラム名（ヘッダー）を直す必要がある。

In [84]:
# fix headers
time.columns = ['発生時（始期）','件数']
time

,発生時（始期）,件数
0,18,44
1,19,39
2,8,37
3,16,36
4,20,33
5,17,32
6,12,28
7,13,25
8,15,24
9,7,23


いよいよ準備万端。この新しいデータフレームでチャートを作ろう。この場合は plotly express の bar charts を参考。

https://plotly.com/python/bar-charts/

In [85]:
fig = px.bar(time,x='発生時（始期）',y='件数')
fig.show()

ありゃ？順番が件数の多い順になってる。実はx軸のオプションっていっぱいあるんだ！その中の一つでカテゴリーの順番を設定できる。

https://plotly.com/python/categorical-axes/

In [86]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数'
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

チャートのテンプレートを変えることでルックスが変わる。

`template`のオプションはこちらから：

```["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white"]```

一つずつ試してみよう。

In [87]:
fig = px.bar(time,
            x='発生時（始期）',
            y='件数',
            template='ggplot2' # change this to see other styles
            )
fig.update_xaxes(categoryorder='category ascending')
fig.show()

### Make your own charts

では、ここで他のチャートを作ってみよう。

例：被害者の年齢、被害者の職業

In [88]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 471 entries, 2776 to 3246
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             471 non-null    object
 1   手口             471 non-null    object
 2   管轄警察署（発生地）     471 non-null    object
 3   管轄交番・駐在所（発生地）  471 non-null    object
 4   市区町村コード（発生地）   471 non-null    int64 
 5   都道府県（発生地）      471 non-null    object
 6   市区町村（発生地）      471 non-null    object
 7   町丁目（発生地）       471 non-null    object
 8   発生年月日（始期）      471 non-null    object
 9   発生時（始期）        471 non-null    object
 10  発生場所           471 non-null    object
 11  発生場所の詳細        471 non-null    object
 12  被害者の年齢         471 non-null    object
 13  被害者の職業         471 non-null    object
 14  施錠関係           471 non-null    object
dtypes: int64(1), object(14)
memory usage: 58.9+ KB


In [89]:
kashiwa['発生場所'].value_counts()

発生場所
その他                    216
その他の住宅（３階建て以下共同住宅等）     91
一戸建住宅                   67
４階建て以上共同住宅              52
駐車（輪）場                  29
道路上                     16
Name: count, dtype: int64

In [90]:
kashiwa['被害者の年齢'].value_counts()

被害者の年齢
10歳代           179
20歳代           123
30歳代            63
40歳代            42
70歳以上           18
50歳代            16
10歳未満           12
60-64歳          10
65-69歳           4
法人・団体、被害者なし      4
Name: count, dtype: int64

In [97]:
kashiwa['被害者の職業'].value_counts()

被害者の職業
その他            257
高校生             89
中学生             51
大学生             46
小学生             24
法人・団体、被害者なし      4
Name: count, dtype: int64

In [98]:
kashiwa['都道府県（発生地）'].value_counts()

都道府県（発生地）
千葉県    471
Name: count, dtype: int64

In [99]:
kashiwa['罪名'].value_counts()

罪名
窃盗    471
Name: count, dtype: int64

In [100]:
kashiwa['手口'].value_counts()

手口
自転車盗    471
Name: count, dtype: int64

In [101]:
kashiwa['発生場所の詳細'].value_counts()

発生場所の詳細
駐車（輪）場    389
その他        82
Name: count, dtype: int64

In [102]:
kashiwa['施錠関係'].value_counts()

施錠関係
施錠せず    305
施錠した    166
Name: count, dtype: int64

In [103]:
kashiwa['管轄交番・駐在所（発生地）'].value_counts()

管轄交番・駐在所（発生地）
柏駅前交番         79
旭町交番          58
南増尾交番         45
南柏駅前交番        39
北柏駅前交番        35
松葉町交番         32
大津ヶ丘交番        28
緑ヶ丘交番         26
柏の葉キャンパス交番    26
高柳交番          23
花野井交番         19
光ヶ丘交番         19
新柏交番          13
増尾駅前交番        13
豊四季駅前交番       10
船戸駐在所          3
泉駐在所           1
富勢駐在所          1
高田原交番          1
Name: count, dtype: int64

In [104]:
kashiwa['管轄警察署（発生地）'].value_counts()

管轄警察署（発生地）
柏    471
Name: count, dtype: int64

In [110]:
fig = px.bar(time,x='被害者の年齢',y='件数')
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['発生時（始期）', '件数'] but received: 被害者の年齢

## その他のチャート（上級編）

複数の変数でチャートを作ると、より深い分析ができる。

In [105]:
fig = px.bar(kashiwa,
            x='発生時（始期）',
            color='施錠関係',
            template='seaborn')
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

↑上のチャートの凡例ののカテゴリーをダブルクリックするとどうなる？

`barmode='group'`を足すと何がどのように変わる？

In [106]:
fig = px.bar(kashiwa,
            x='発生時（始期）',
            color='施錠関係',
            barmode='group', # group the categories,
            template='seaborn'
            )
fig.update_traces(
    marker_line_width=0
)
fig.update_xaxes(categoryorder='category ascending')
fig.show()

# Python Lesson: Functions 関数

突然だが、ちょっとここでPythonのレッスンを投入！次のステップでFunction (関数)を使うので、まずは簡単に学ぼう。

関数とは、特定のタスクを実行するために組織化され<span style="font-size:2em">【再利用可能】</span>なコードのブロックです。例えば、繰り返し同じ作業をするぞ！と分かったら、同じコードブロックをコピペするよりかは関数を作った方が良い。


In [111]:
# example of a simple function
def hello():
    print('こんにちは😀')

In [112]:
# call the function
hello()

こんにちは😀


In [113]:
# example of a function with an argument
def hello(name):
    print(f'✨こんにちは {name}✨!')

In [114]:
# call the function
hello('Misuzu')

✨こんにちは Misuzu✨!


In [115]:
def add(a,b):
    return(a+b)

In [116]:
add(100,55)

155

## チャレンジ問題

メジャーリーグの野球の試合でピッチャーの時速が画面で表示されるが、これは大体MPH (miles per hour)なので、日本人には分かりにくい。

<img src="https://raw.githubusercontent.com/yohman/24-1-Reitaku-GIS/229177aa00887ceccb2a6d0cf2b0cd1263298cc4/Weeks/Week08/images/mlb%20mpg.jpeg" width=600>

では、このMilesをKilometersに変える関数を書きましょう！計算は：

`1 miles = 1.60934 kilometers`

In [121]:
# create the function
def mtokm(miles):
    return (miles * 1.60934)

In [118]:
# call the function
mtokm(104)

167.37136

# Geocoding

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/Geocoding_01.png" width=400>

住所だけではマップイングできません。座標が必要です。なので、住所から座標を特定するプロセスが必要である。このプロセスを<h1>【ジオコーティング】</h1>という。


ジオコーティングと言えば、色んな方法があります。現在、日本で無料でジオコーティングサービスを提供しているのが国土地理院のジオコーティングAPI。

試してみよう。このようにURLをブラウザーで記入するだけで座標が返ってくるサービスである。国土地理院さん、とても便利なサービス、ありがとうございます！

https://msearch.gsi.go.jp/address-search/AddressSearch?q=麗澤大学

では、このプロセスに従って、アドレスから座標を出力する Python 関数を作成します。

In [122]:
# 関数を作成
def geocode(address):

    # ジオコーティングURL
    url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

    # try/exceptでエラーをキャッチ
    # 成功の場合
    try:
        s_quote = urllib.parse.quote(address)
        response = requests.get(url + s_quote)
        if len(response.json())>0:
            return response.json()[0]["geometry"]["coordinates"] 
        else:
            return False
        
    # 失敗の場合
    except:
        return False

関数の使い方は簡単！

In [123]:
geocode('南柏')

[139.953476, 35.845783]

In [124]:
geocode('我孫子')

[140.028336, 35.864166]

In [125]:
geocode('埼玉県越谷市大里328-3')

[139.784546, 35.916019]

## データフレームの準備

`kashiwa`のデータフレームの各行の住所をジオコーティングする前に`住所`と座標（`lat`,`lon`）のフィールドを作りましょう。

In [126]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 471 entries, 2776 to 3246
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             471 non-null    object
 1   手口             471 non-null    object
 2   管轄警察署（発生地）     471 non-null    object
 3   管轄交番・駐在所（発生地）  471 non-null    object
 4   市区町村コード（発生地）   471 non-null    int64 
 5   都道府県（発生地）      471 non-null    object
 6   市区町村（発生地）      471 non-null    object
 7   町丁目（発生地）       471 non-null    object
 8   発生年月日（始期）      471 non-null    object
 9   発生時（始期）        471 non-null    object
 10  発生場所           471 non-null    object
 11  発生場所の詳細        471 non-null    object
 12  被害者の年齢         471 non-null    object
 13  被害者の職業         471 non-null    object
 14  施錠関係           471 non-null    object
dtypes: int64(1), object(14)
memory usage: 58.9+ KB


In [127]:
# 空の住所フィールドを作成
kashiwa['住所'] = ''

In [128]:
kashiwa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 471 entries, 2776 to 3246
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   罪名             471 non-null    object
 1   手口             471 non-null    object
 2   管轄警察署（発生地）     471 non-null    object
 3   管轄交番・駐在所（発生地）  471 non-null    object
 4   市区町村コード（発生地）   471 non-null    int64 
 5   都道府県（発生地）      471 non-null    object
 6   市区町村（発生地）      471 non-null    object
 7   町丁目（発生地）       471 non-null    object
 8   発生年月日（始期）      471 non-null    object
 9   発生時（始期）        471 non-null    object
 10  発生場所           471 non-null    object
 11  発生場所の詳細        471 non-null    object
 12  被害者の年齢         471 non-null    object
 13  被害者の職業         471 non-null    object
 14  施錠関係           471 non-null    object
 15  住所             471 non-null    object
dtypes: int64(1), object(15)
memory usage: 62.6+ KB


In [129]:
# 空だがfloatとしてフィールドを作成
kashiwa['lat'] = pd.Series(dtype=float)
kashiwa['lon'] = pd.Series(dtype=float)

<img src="https://raw.githubusercontent.com/yohman/23-1-Reitaku-GIS/main/Weeks/Week08/images/concat.png" width=600>

↑で作った住所フィールドに次の 3 つのフィールドを連結します。

1. 都道府県（発生地）
1. 市区町村（発生地）
1. 町丁目（発生地）

In [131]:
# 住所フィールドを作成
kashiwa['住所'] = kashiwa['都道府県（発生地）']+kashiwa['市区町村（発生地）']+kashiwa['町丁目（発生地）']

In [132]:
# random sampleで上手く行ったかどうかをチェック
kashiwa[['都道府県（発生地）','市区町村（発生地）','町丁目（発生地）','住所']].sample(5)


,都道府県（発生地）,市区町村（発生地）,町丁目（発生地）,住所
2878,千葉県,柏市,豊四季,千葉県柏市豊四季
2874,千葉県,柏市,大井,千葉県柏市大井
2829,千葉県,柏市,根戸,千葉県柏市根戸
2788,千葉県,柏市,西山２丁目,千葉県柏市西山２丁目
2885,千葉県,柏市,南柏１丁目,千葉県柏市南柏１丁目


これで準備は整えました。`kashiwa` のデータフレームを for loop に入れて住所を一つずつジオコーティングしよう。

この作業は数分かかるので注意。千件以上のデータフレームはなるべく避けよう。

In [134]:
# kashiwaデータフレームをループ
for i,row in kashiwa.iterrows():

    # ジオコーティング成功
    if geocode(row['住所']) != False:
        # 座標を変数に
        lon = geocode(row['住所'])[0]
        lat = geocode(row['住所'])[1]

        # データフレームに値をインプット        
        kashiwa.loc[i,'lon'] = lon
        kashiwa.loc[i,'lat'] = lat

        # 結果をprint
        print(row['住所'],lon,lat)
    
    # ジオコーティング失敗
    else:
        print(row['住所'],'ジオコーティング失敗')
        continue

千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市今谷上町 139.956406 35.841019
千葉県柏市酒井根 139.957825 35.81815
千葉県柏市酒井根３丁目 139.965973 35.818859
千葉県柏市酒井根４丁目 139.960007 35.820488
千葉県柏市酒井根５丁目 139.961349 35.824585
千葉県柏市酒井根５丁目 139.961349 35.824585
千葉県柏市逆井 139.982468 35.815441
千葉県柏市中新宿３丁目 139.946884 35.83292
千葉県柏市西山２丁目 139.95401 35.82056
千葉県柏市光ケ丘１丁目 139.957794 35.833195
千葉県柏市光ケ丘１丁目 139.957794 35.833195
千葉県柏市光ケ丘団地 139.957809 35.829479
千葉県柏市東中新宿１丁目 139.960556 35.833984
千葉県柏市東中新宿１丁目 139.960556 35.833984
千葉県柏市東中新宿１丁目 139.960556 35.833984
千葉県柏市柏 139.986511 35.864445
千葉県柏市柏 139.986511 35.864445
千葉県柏市柏 139.986511 35.864445
千葉県柏市柏下 139.983795 35.870007
千葉県柏市柏堀之内新田 139.983582 35.875332
千葉県柏市北柏２丁目 139.989914 35.872643
千葉県柏市北柏２丁目 139.989914 35.872643
千葉県柏市北柏２丁目 139.989914 35.872643
千葉県柏市北柏３丁目 139.987793 35.872192
千葉県柏市北柏３丁目 139.987793 35.872192
千葉県柏市北柏４丁目 139.986893 35.870586
千葉県柏市北柏４丁目 139.986893 35.870586
千葉県柏市北柏５丁目 139.990295 3

In [135]:
# check your output, show address and lat/lon for 5 samples
kashiwa[['住所','lat','lon']].sample(5)


,住所,lat,lon
2881,千葉県柏市豊四季,35.859928,139.949814
3110,千葉県柏市柏２丁目,35.861694,139.973999
2794,千葉県柏市東中新宿１丁目,35.833984,139.960556
2900,千葉県柏市南柏中央,35.844471,139.954956
3145,千葉県柏市桜台,35.860554,139.981522


In [136]:
# geocode出来なかったデータ、latがNaNを削除
kashiwa = kashiwa[kashiwa['lat'].notna()]


## Let's map it!

待ってました、いよいよマップタイム！

In [ ]:
# 全データの中央座標
center_lat = kashiwa.lat.mean()
center_lon = kashiwa.lon.mean() 

In [ ]:
# make the map
m = folium.Map(location=[center_lat,center_lon], 
               zoom_start=12)

# マーカーがメチャクチャ多い場合はclusterで処理！
marker_cluster = plugins.MarkerCluster().add_to(m)
 
# kashiwaのデータフレームをループしてマーカーを作る
for index, row in kashiwa.iterrows():
    latlon = [row['lat'],row['lon']]
    folium.Marker(latlon, 
                  tooltip=row['被害者の年齢'],
                ).add_to(marker_cluster) # mapにではなくmarker_clusterに足す

# show the map
m


## Heatmap

Foliumのエキストラ機能としてヒートマップが作れる！では、やって見よう。

まずはデータをヒートマップが必要とする形式を作る。

In [ ]:
# make a list of lat/lon's
heatmap_lat_lon = kashiwa[['lat','lon']].values.tolist()

# view the list
heatmap_lat_lon

In [ ]:
from folium.plugins import HeatMap

# make the map
m = folium.Map(location=[center_lat,center_lon], 
               zoom_start=12,
               tiles='cartodbdark_matter')

# add the heatmap
HeatMap(
    data = heatmap_lat_lon,
    radius=15,    
).add_to(m)

# マーカーがメチャクチャ多い場合はclusterで処理！
marker_cluster = plugins.MarkerCluster().add_to(m)

# kashiwaのデータフレームをループしてマーカーを作る
for index, row in kashiwa.iterrows():
    latlon = [row['lat'],row['lon']]
    folium.Marker(latlon, 
                  tooltip=row['被害者の年齢'],
                ).add_to(marker_cluster) # mapにではなくmarker_clusterに足す

# show the map
m

# 今日の課題

**DUE：日曜日の深夜12時まで！**

違う場所の犯罪マップ作りに挑戦。

このサイトから都道府県を選び、データをダウンロード：

https://www.npa.go.jp/toukei/seianki/hanzaiopendatalink.html

注意！ジオコーティングは時間がかかるので、データを千件以下に絞ってからジオコーティングをするように。